<a href="https://colab.research.google.com/github/XushengShangguan/DS4200/blob/main/rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/NeyoNought47/CS-4120-Project-Repo.git

Cloning into 'CS-4120-Project-Repo'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 9 (delta 0), reused 9 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (9/9), done.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

data_path = '/content/drive/My Drive/clean'

if os.path.exists(data_path):
    print(os.listdir(data_path))
    print("The file exists")
else:
    print(f"{data_path} does not exist")

['.DS_Store', 'es-SV', 'es-PE', 'es-NI', 'es-DO', 'es-EC', 'es-PA', 'es-PR', 'es-UY', 'es-CO', 'es-CR', 'es-VE', 'es-AR', 'es-HN', 'es-CL', 'std_es']
The file exists


In [4]:
import os

region_data = {}

if os.path.exists(data_path):
    sub_folders = sorted(os.listdir(data_path))

    for folder_name in sub_folders:
        folder_full_path = os.path.join(data_path, folder_name)

        if os.path.isdir(folder_full_path) and folder_name.startswith('es-'):
            path_en = os.path.join(folder_full_path, 'all.en')
            path_es = os.path.join(folder_full_path, 'all.es')

            if os.path.exists(path_en) and os.path.exists(path_es):
                with open(path_en, 'r', encoding='utf-8') as f:
                    lines_en = f.read().strip().split('\n')
                with open(path_es, 'r', encoding='utf-8') as f:
                    lines_es = f.read().strip().split('\n')

                current_pairs = []
                if len(lines_en) == len(lines_es):
                    for en, es in zip(lines_en, lines_es):
                        if en.strip() and es.strip():
                            current_pairs.append([en.strip(), es.strip()])

                    region_data[folder_name] = current_pairs


print("regions:", list(region_data.keys()))

regions: ['es-AR', 'es-CL', 'es-CO', 'es-CR', 'es-DO', 'es-EC', 'es-HN', 'es-NI', 'es-PA', 'es-PE', 'es-PR', 'es-SV', 'es-UY', 'es-VE']


In [5]:
import re

SOS_token = 0
EOS_token = 1
MAX_LENGTH = 20

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

def normalizeString(s):
    s = s.lower().strip()
    s = re.sub(r"([.!?¿¡,])", r" \1", s)
    s = re.sub(r"[^a-zA-ZáéíóúñÁÉÍÓÚÑ.!?¿¡,]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
           len(p[1].split(' ')) < MAX_LENGTH

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")

def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair, input_lang, output_lang):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

device: cpu


In [7]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.dropout = nn.Dropout(dropout_p)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [8]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length
        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)

        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)

        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)

        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)

        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)

        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [9]:
def validate(encoder, decoder, validation_pairs, input_lang, output_lang, criterion):
    total_loss = 0
    total_correct_tokens = 0
    total_steps = len(validation_pairs)

    encoder.eval()
    decoder.eval()

    with torch.no_grad():
        for pair in validation_pairs:
            input_tensor = tensorFromSentence(input_lang, pair[0])
            target_tensor = tensorFromSentence(output_lang, pair[1])

            input_length = input_tensor.size(0)
            target_length = target_tensor.size(0)

            encoder_hidden = encoder.initHidden()
            encoder_outputs = torch.zeros(MAX_LENGTH, encoder.hidden_size, device=device)

            for ei in range(input_length):
                encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
                encoder_outputs[ei] = encoder_output[0, 0]

            decoder_input = torch.tensor([[SOS_token]], device=device)
            decoder_hidden = encoder_hidden

            for di in range(target_length):
                decoder_output, decoder_hidden, _ = decoder(
                    decoder_input, decoder_hidden, encoder_outputs)

                loss = criterion(decoder_output, target_tensor[di])
                total_loss += loss.item()

                topv, topi = decoder_output.topk(1)
                if topi.item() == target_tensor[di].item():
                    total_correct_tokens += 1

                decoder_input = topi.squeeze().detach()
                if decoder_input.item() == EOS_token:
                    break

    encoder.train()
    decoder.train()

    avg_loss = total_loss / (total_steps * target_length)
    avg_acc = total_correct_tokens / (total_steps * target_length)
    return avg_loss, avg_acc

In [10]:
import random

def train_step(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0
    correct_tokens = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < 0.5 else False

    if use_teacher_forcing:
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)

            topv, topi = decoder_output.topk(1)
            if topi.item() == target_tensor[di].item():
                correct_tokens += 1

            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di] # Teacher forcing

    else:
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)

            topv, topi = decoder_output.topk(1)
            if topi.item() == target_tensor[di].item():
                correct_tokens += 1

            decoder_input = topi.squeeze().detach()
            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length, correct_tokens / target_length

In [11]:
from torch import optim
from tqdm.notebook import tqdm

def train_specific_region(region_name, n_epochs=5, learning_rate=0.001):

    if region_name not in region_data:
        print(f"Error: region is not in the list")

    raw_pairs = region_data[region_name]

    input_lang = Lang("eng")
    output_lang = Lang("spa")

    clean_pairs = []
    for en, es in raw_pairs:
        clean_en = normalizeString(en)
        clean_es = normalizeString(es)

        if len(clean_en.split()) < MAX_LENGTH and len(clean_es.split()) < MAX_LENGTH:
            clean_pairs.append([clean_en, clean_es])
            input_lang.addSentence(clean_en)
            output_lang.addSentence(clean_es)

    random.shuffle(clean_pairs)
    val_split = int(len(clean_pairs) * 0.8)
    train_pairs_local = clean_pairs[:val_split]
    val_pairs_local = clean_pairs[val_split:]

    hidden_size = 256
    encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
    decoder = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    steps_per_epoch = 10000

    for epoch in range(1, n_epochs + 1):
        epoch_loss = 0
        epoch_acc = 0

        with tqdm(total=steps_per_epoch, unit="step", desc=f"Epoch {epoch}") as pbar:
            for i in range(steps_per_epoch):
                pair = random.choice(train_pairs_local)

                input_tensor = tensorFromSentence(input_lang, pair[0])
                target_tensor = tensorFromSentence(output_lang, pair[1])
                loss, acc = train_step(input_tensor, target_tensor, encoder,
                                  decoder, encoder_optimizer, decoder_optimizer, criterion)

                epoch_loss += loss
                epoch_acc += acc

                pbar.set_postfix({'acc': f'{epoch_acc/(i+1):.3f}', 'loss': f'{epoch_loss/(i+1):.3f}'})
                pbar.update(1)


            val_sample = random.sample(val_pairs_local, min(len(val_pairs_local), 100))

            val_loss, val_acc = validate(encoder, decoder, val_sample, input_lang, output_lang, criterion)

            pbar.set_postfix_str(
                f"acc: {epoch_acc/steps_per_epoch:.3f} - "
                f"loss: {epoch_loss/steps_per_epoch:.3f} - "
                f"val_acc: {val_acc:.3f} - "
                f"val_loss: {val_loss:.3f}"
            )

    return encoder, decoder, input_lang, output_lang

In [12]:
target_region = 'es-SV'

model_en, model_de, lang_in, lang_out = train_specific_region(
    target_region,
    n_epochs=5,
    learning_rate=0.001
)

Epoch 1:   0%|          | 0/10000 [00:00<?, ?step/s]

KeyboardInterrupt: 

In [ ]:
target_region = 'es-PE'

model_en, model_de, lang_in, lang_out = train_specific_region(
    target_region,
    n_epochs=5,
    learning_rate=0.001
)

In [ ]:
target_region = 'es-NI'

model_en, model_de, lang_in, lang_out = train_specific_region(
    target_region,
    n_epochs=5,
    learning_rate=0.001
)

In [ ]:
target_region = 'es-DO'

model_en, model_de, lang_in, lang_out = train_specific_region(
    target_region,
    n_epochs=5,
    learning_rate=0.001
)

In [ ]:
target_region = 'es-EC'

model_en, model_de, lang_in, lang_out = train_specific_region(
    target_region,
    n_epochs=5,
    learning_rate=0.001
)

In [ ]:
target_region = 'es-PA'

model_en, model_de, lang_in, lang_out = train_specific_region(
    target_region,
    n_epochs=5,
    learning_rate=0.001
)

In [ ]:
target_region = 'es-PR'

model_en, model_de, lang_in, lang_out = train_specific_region(
    target_region,
    n_epochs=5,
    learning_rate=0.001
)

In [ ]:
target_region = 'es-UY'

model_en, model_de, lang_in, lang_out = train_specific_region(
    target_region,
    n_epochs=5,
    learning_rate=0.001
)

In [ ]:
target_region = 'es-CO'

model_en, model_de, lang_in, lang_out = train_specific_region(
    target_region,
    n_epochs=5,
    learning_rate=0.001
)

In [ ]:
target_region = 'es-CR'

model_en, model_de, lang_in, lang_out = train_specific_region(
    target_region,
    n_epochs=5,
    learning_rate=0.001
)

In [ ]:
target_region = 'es-VE'

model_en, model_de, lang_in, lang_out = train_specific_region(
    target_region,
    n_epochs=5,
    learning_rate=0.001
)

In [ ]:
target_region = 'es-AR'

model_en, model_de, lang_in, lang_out = train_specific_region(
    target_region,
    n_epochs=5,
    learning_rate=0.001
)

In [ ]:
target_region = 'es-HN'

model_en, model_de, lang_in, lang_out = train_specific_region(
    target_region,
    n_epochs=5,
    learning_rate=0.001
)

In [ ]:
target_region = 'es-CL'

model_en, model_de, lang_in, lang_out = train_specific_region(
    target_region,
    n_epochs=5,
    learning_rate=0.001
)